In [511]:
import capytaine as cpt
import numpy as np
import matplotlib.pyplot as plt
from capytaine.bem.airy_waves import airy_waves_potential, airy_waves_velocity, froude_krylov_force



In [512]:
#%%writefile pwa_utils.py
def generate_body(xyz):
    mesh1 = cpt.meshes.predefined.mesh_sphere(radius=2,center=(xyz[0],xyz[1],xyz[2]))
    body = cpt.FloatingBody(mesh1)
    body.add_translation_dof(name='Heave')
    body = body.immersed_part()
    body.name = f'{xyz[0]}_{xyz[1]}_{xyz[2]}'
    return body


def get_results(problems):
    results = [solver.solve(pb, keep_details = True) for pb in sorted(problems)]
    return results


#calculate angle theta_ij from centre of one body to other
def theta_ij(X,Y): 
    x1,y1= X[0],X[1]
    x2,y2 = Y[0], Y[1]
    
    if x1 ==x2 and y1==y2:
        return 0
    if x2==x1:
        theta = np.pi/2
    else:
        theta = np.arctan((y2-y1)/(x2-x1))
    return theta


#step 2
def phi_j_star(phi_ij,theta,X,Y,z,k):
    
    '''phi_ij is the vector of all the effect at that body from all other bodies'''
    x,y = X[0],X[1]
    xj,yj = Y[0],Y[1]
    if x==xj and y==yj:
        return 0
    multiplier = np.exp((1j*k*(-1*np.abs(x-xj))*np.cos(theta)) + ((-1*np.abs(y-yj))*np.sin(theta)))
    print(f'the multiplier for {X}{Y} and {theta} is {multiplier}')
    res = phi_ij * multiplier #kz = 0 #e^kz = 1
    return res

#{(10, 10, 0): {(10, 10, 0): 0,
  #(0, 0, 0): (8.415476709952118-2.9519008598532284j),
  #(5, 5, 0): (8.415476709952118-2.9519008598532284j),
  #(15, 15, 0): (8.415476709952118-2.9519008598532284j)

def get_phistarj_sum(phi_starj,xyzees):
    xyz_phi = {xyz :[] for xyz in xyzees}
    for k,v in phi_starj.items():
        for s,m in v.items():
            print(f"thee value of m = {m}")
            xyz_phi[k].append(m)
        print("next xyz")
    print(xyz_phi.items())
    xyz_phi = {k:sum(v) for k,v in xyz_phi.items()}
    print("Afteer summation ")
    print(xyz_phi.items())
    return xyz_phi

p = 0
omega = 1.1
rho = 850 # density of our special material
wave_amp = 1
wave_num =  1.0/9.81

In [513]:
from capytaine.bem.problems_and_results import *



def airy_waves_velocity_iter(points, pb):
    """Compute the fluid velocity for Airy waves at a given point (or array of points).
    Parameters
    ----------
    points: array of shape (3) or (N x 3)
        coordinates of the points in which to evaluate the potential.
    pb: DiffractionProblem
        problem with the environmental conditions (g, rho, ...) of interest
    Returns
    -------
    array of shape (3) or (N x 3)
        the velocity vectors
    """

    x, y, z = points.T
    k = pb.wavenumber
    h = pb.depth

    wbar = x * np.cos(pb.wave_direction) + y * np.sin(pb.wave_direction)

    if 0 <= k*h < 20:
        cih = np.cosh(k*(z+h))/np.cosh(k*h)
        sih = np.sinh(k*(z+h))/np.cosh(k*h)
    else:
        cih = np.exp(k*z)
        sih = np.exp(k*z)

    v = pb.g*k/pb.omega * \
        np.exp(1j * k * wbar) * \
        np.array([np.cos(pb.wave_direction) * cih, np.sin(pb.wave_direction) * cih, -1j * sih])

    return v.T






class DiffractionProblemIter(LinearPotentialFlowProblem):
    """Particular LinearPotentialFlowProblem with boundary conditions
    computed from an incoming Airy wave."""
    
    
    _default_parameters = {'rho': 1000.0, 'g': 9.81, 'omega': 1.0,
                      'free_surface': 0.0, 'water_depth': np.infty,
                      'wave_direction': 0.0}

    def __init__(self, *,
                 body=None,
                 free_surface=_default_parameters['free_surface'],
                 sea_bottom=-_default_parameters['water_depth'],
                 omega=None, period=None, wavenumber=None, wavelength=None,
                 rho=_default_parameters['rho'],
                 g=_default_parameters['g'],
                 wave_direction=_default_parameters['wave_direction']):

        self.wave_direction = float(wave_direction)

        super().__init__(body=body, free_surface=free_surface, sea_bottom=sea_bottom,
                         omega=omega, period=period, wavenumber=wavenumber, wavelength=wavelength, rho=rho, g=g)

        if not (-2*np.pi-1e-3 <= self.wave_direction <= 2*np.pi+1e-3):
            LOG.warning(f"The value {self.wave_direction} has been provided for the wave direction, and it does not look like an angle in radians. "
                         "The wave direction in Capytaine is defined in radians and not in degrees, so the result might not be what you expect. "
                         "If you were actually giving an angle in radians, use the modulo operator to give a value between -2π and 2π to disable this warning.")

        if self.body is not None:

            self.boundary_condition = -(
                    airy_waves_velocity_iter(self.body.mesh.faces_centers, self)
                    * self.body.mesh.faces_normals
            ).sum(axis=1)

            if len(self.body.dofs) == 0:
                LOG.warning(f"The body {self.body.name} used in diffraction problem has no dofs!")

    def _astuple(self):
        return super()._astuple() + (self.wave_direction,)

    def _asdict(self):
        d = super()._asdict()
        d["wave_direction"] = self.wave_direction
        return d

    def _str_other_attributes(self):
        return [f"wave_direction={self.wave_direction:.3f}"]

    def make_results_container(self, *args, **kwargs):
        return DiffractionResult(self, *args, **kwargs)



The method is based upon an idea due to Simon 3
originally devised in connection with the theory of arrays
of wave-power devices. A diverging wave scattered from
one cylinder is replaced by a plane wave of appropriate
amplitude in the neighbourhood of another cylinder. Once
the amplitude and phase of the equivalent plane wave have
been determined, **the problem reduces to summing the
effects of plane waves on any given cylinder**

#### 1. Initialize the bodies and define diffraction and radiation problems and solve

In [542]:
xyzees = {(0,0,0),(5,5,0),(10,10,0),(15,15,0)}
    
bodies = [generate_body(xyz) for xyz in xyzees ]

neighbors = {(0,0,0):[(5,5,0),(10,10,0),(15,15,0)],  #so bad..need to write a funky func for it
            (10,10,0):[(0,0,0),(5,5,0),(15,15,0)],
             (5,5,0):[(0,0,0),(10,10,0),(15,15,0)],
             (15,15,0):[(0,0,0),(10,10,0),(5,5,0)]     
            }



def get_neighbors(xyzees):
    neighbor = {xyz:[] for xyz in xyzees}
    for xyz in xyzees:
        for zyx in xyzees:
            if not xyz == zyx:
                neighbor[xyz].append(zyx)
    return neighbor


print(get_neighbors(xyzees))
loc_bodies = {body:xyz for xyz,body in zip(xyzees,bodies)}
loc_to_body = {xyz:body for xyz,body in zip(xyzees,bodies)}
solver = cpt.BEMSolver()


diff_problems = {body:cpt.DiffractionProblem(body=body, sea_bottom=-np.infty,
                                      omega=omega, wave_direction=0.) for body in bodies}

diff_loc= {generate_body(loc):loc for loc in xyzees }

loc_diff = {loc_bodies.get(body):diff for body,diff in diff_problems.items() }

rad_problems = {body: cpt.RadiationProblem(body=body, sea_bottom=-np.infty,
                                      omega=omega) for body in bodies}

diff_results = {body:solver.solve(problem) for body,problem in diff_problems.items()}
rad_results = {body:solver.solve(problem) for body,problem in rad_problems.items()}
diff_results

{(10, 10, 0): [(0, 0, 0), (5, 5, 0), (15, 15, 0)], (0, 0, 0): [(10, 10, 0), (5, 5, 0), (15, 15, 0)], (5, 5, 0): [(10, 10, 0), (0, 0, 0), (15, 15, 0)], (15, 15, 0): [(10, 10, 0), (0, 0, 0), (5, 5, 0)]}


{FloatingBody(mesh=sphere_2202, dofs={Heave}, name=10_10_0): <capytaine.bem.problems_and_results.DiffractionResult at 0x13dabab10>,
 FloatingBody(mesh=sphere_2208, dofs={Heave}, name=0_0_0): <capytaine.bem.problems_and_results.DiffractionResult at 0x13d9ebc50>,
 FloatingBody(mesh=sphere_2214, dofs={Heave}, name=5_5_0): <capytaine.bem.problems_and_results.DiffractionResult at 0x13da4d590>,
 FloatingBody(mesh=sphere_2220, dofs={Heave}, name=15_15_0): <capytaine.bem.problems_and_results.DiffractionResult at 0x13da4b850>}

### 2. Get the Incident potentials, diffraction potentials, radiation potentials for each body at their own location and the location of other bodies
Currently only incident potential can be generated at any other location from capytaine. But diffraction and radiation are available for the mesh
However, according to simon(1982) we can approximate the impact of outgoing waves from a body on all other bodies by an incident plane wave of appropriately chosen amplitude.

#### The diffraction potential depends on the boundary condition from the incoming airy waves so that has to be updated for new iteration.
1) From first iteration, we calculate effect at other locations
2) Next, we define new diffraction problem for each body with updated incident potentials.
3) We add all the potentials for total potentials and repeat until the max iteration


#### find the location of neighbors to each floating body

In [515]:
body_neighbors_locs = {body:neighbors.get(loc_bodies.get(body)) for body in bodies}
body_neighbors_locs

{FloatingBody(mesh=sphere_1776, dofs={Heave}, name=10_10_0): [(0, 0, 0),
  (5, 5, 0),
  (15, 15, 0)],
 FloatingBody(mesh=sphere_1782, dofs={Heave}, name=0_0_0): [(5, 5, 0),
  (10, 10, 0),
  (15, 15, 0)],
 FloatingBody(mesh=sphere_1788, dofs={Heave}, name=5_5_0): [(0, 0, 0),
  (10, 10, 0),
  (15, 15, 0)],
 FloatingBody(mesh=sphere_1794, dofs={Heave}, name=15_15_0): [(0, 0, 0),
  (10, 10, 0),
  (5, 5, 0)]}

###  Get the potential of a body to other location

In [516]:
body_potential_at_neighbors = {body:(dict(zip(body_neighbors_locs[body], 
                                      airy_waves_potential(np.array(body_neighbors_locs[body]),diff_problems[body])))) for body in bodies}
body_potential_at_neighbors

{FloatingBody(mesh=sphere_1776, dofs={Heave}, name=10_10_0): {(0,
   0,
   0): -8.918181818181818j,
  (5, 5, 0): (5.1579248911132085-7.275285407445096j),
  (15, 15, 0): (8.57245113234644+2.459074729584482j)},
 FloatingBody(mesh=sphere_1782, dofs={Heave}, name=0_0_0): {(5,
   5,
   0): (5.1579248911132085-7.275285407445096j),
  (10, 10, 0): (8.415476709952118-2.9519008598532284j),
  (15, 15, 0): (8.57245113234644+2.459074729584482j)},
 FloatingBody(mesh=sphere_1788, dofs={Heave}, name=5_5_0): {(0,
   0,
   0): -8.918181818181818j,
  (10, 10, 0): (8.415476709952118-2.9519008598532284j),
  (15, 15, 0): (8.57245113234644+2.459074729584482j)},
 FloatingBody(mesh=sphere_1794, dofs={Heave}, name=15_15_0): {(0,
   0,
   0): -8.918181818181818j,
  (10, 10, 0): (8.415476709952118-2.9519008598532284j),
  (5, 5, 0): (5.1579248911132085-7.275285407445096j)}}

### $\phi_{ij}$ Get the all other potential at each location/body

In [517]:
# def get_all_other_phi(body_potential_at_neighbors):
all_other_phi_each_loc = {xyz:{loc_bodies.get(d):k.get(xyz,0) for d,k in body_potential_at_neighbors.items()} for xyz in xyzees}
all_other_phi_each_loc

{(10, 10, 0): {(10, 10, 0): 0,
  (0, 0, 0): (8.415476709952118-2.9519008598532284j),
  (5, 5, 0): (8.415476709952118-2.9519008598532284j),
  (15, 15, 0): (8.415476709952118-2.9519008598532284j)},
 (0, 0, 0): {(10, 10, 0): -8.918181818181818j,
  (0, 0, 0): 0,
  (5, 5, 0): -8.918181818181818j,
  (15, 15, 0): -8.918181818181818j},
 (5, 5, 0): {(10, 10, 0): (5.1579248911132085-7.275285407445096j),
  (0, 0, 0): (5.1579248911132085-7.275285407445096j),
  (5, 5, 0): 0,
  (15, 15, 0): (5.1579248911132085-7.275285407445096j)},
 (15, 15, 0): {(10, 10, 0): (8.57245113234644+2.459074729584482j),
  (0, 0, 0): (8.57245113234644+2.459074729584482j),
  (5, 5, 0): (8.57245113234644+2.459074729584482j),
  (15, 15, 0): 0}}

### $\phi_j^*$ get the total effect of each bodies

In [518]:
#step 2

thetas = {k:{s:theta_ij(k,s) for s,m in v.items()} for k,v in all_other_phi_each_loc.items()}
thetas

{(10, 10, 0): {(10, 10, 0): 0,
  (0, 0, 0): 0.7853981633974483,
  (5, 5, 0): 0.7853981633974483,
  (15, 15, 0): 0.7853981633974483},
 (0, 0, 0): {(10, 10, 0): 0.7853981633974483,
  (0, 0, 0): 0,
  (5, 5, 0): 0.7853981633974483,
  (15, 15, 0): 0.7853981633974483},
 (5, 5, 0): {(10, 10, 0): 0.7853981633974483,
  (0, 0, 0): 0.7853981633974483,
  (5, 5, 0): 0,
  (15, 15, 0): 0.7853981633974483},
 (15, 15, 0): {(10, 10, 0): 0.7853981633974483,
  (0, 0, 0): 0.7853981633974483,
  (5, 5, 0): 0.7853981633974483,
  (15, 15, 0): 0}}

#### Make sure we add all N but not when i==j so the value should be 0 for same location 

In [519]:
z = 0
phi_starj = {xyz:{nbros:phi_j_star(all_other_phi_each_loc[xyz][nbros],thetas[xyz][nbros],nbros,xyz,z,wave_num) for nbros in neighbors} for xyz in xyzees}
phi_starj

the multiplier for (0, 0, 0)(10, 10, 0) and 0.7853981633974483 is (0.0006380785685019252-0.0005605442829208494j)
the multiplier for (5, 5, 0)(10, 10, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (15, 15, 0)(10, 10, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (10, 10, 0)(0, 0, 0) and 0.7853981633974483 is (0.0006380785685019252-0.0005605442829208494j)
the multiplier for (5, 5, 0)(0, 0, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (15, 15, 0)(0, 0, 0) and 0.7853981633974483 is (1.164007847899644e-05-2.1844294383065847e-05j)
the multiplier for (0, 0, 0)(5, 5, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (10, 10, 0)(5, 5, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (15, 15, 0)(5, 5, 0) and 0.7853981633974483 is (0.0006380785685019252-0.0005605442829208494j)
the 

{(10, 10, 0): {(0, 0, 0): (0.003715064181607672-0.006600792032831968j),
  (10, 10, 0): 0,
  (5, 5, 0): (0.199159979590679-0.16698965552300543j),
  (15, 15, 0): (0.199159979590679-0.16698965552300543j)},
 (0, 0, 0): {(0, 0, 0): 0,
  (10, 10, 0): (-0.004999035832230484-0.005690500688185351j),
  (5, 5, 0): (-0.09165512192050691-0.24320686903948394j),
  (15, 15, 0): (-0.00019481138899806906-0.00010380833625359552j)},
 (5, 5, 0): {(0, 0, 0): (0.06589073921823238-0.25141331111841825j),
  (10, 10, 0): (0.06589073921823238-0.25141331111841825j),
  (5, 5, 0): 0,
  (15, 15, 0): (-0.0007869583107988657-0.007533449007674166j)},
 (15, 15, 0): {(0, 0, 0): (0.00015350075624087593-0.0001586353232813415j),
  (10, 10, 0): (0.25905121034128703-0.021040744829162822j),
  (5, 5, 0): (0.006848317628024039-0.0032361555895626336j),
  (15, 15, 0): 0}}

### Get the Total potential on the body surface

In [520]:


new_excitation = get_phistarj_sum(phi_starj,xyzees)
new_excitation     

thee value of m = (0.003715064181607672-0.006600792032831968j)
thee value of m = 0
thee value of m = (0.199159979590679-0.16698965552300543j)
thee value of m = (0.199159979590679-0.16698965552300543j)
next xyz
thee value of m = 0
thee value of m = (-0.004999035832230484-0.005690500688185351j)
thee value of m = (-0.09165512192050691-0.24320686903948394j)
thee value of m = (-0.00019481138899806906-0.00010380833625359552j)
next xyz
thee value of m = (0.06589073921823238-0.25141331111841825j)
thee value of m = (0.06589073921823238-0.25141331111841825j)
thee value of m = 0
thee value of m = (-0.0007869583107988657-0.007533449007674166j)
next xyz
thee value of m = (0.00015350075624087593-0.0001586353232813415j)
thee value of m = (0.25905121034128703-0.021040744829162822j)
thee value of m = (0.006848317628024039-0.0032361555895626336j)
thee value of m = 0
next xyz
dict_items([((10, 10, 0), [(0.003715064181607672-0.006600792032831968j), 0, (0.199159979590679-0.16698965552300543j), (0.199159979

{(10, 10, 0): (0.40203502336296565-0.34058010307884284j),
 (0, 0, 0): (-0.09684896914173546-0.24900117806392288j),
 (5, 5, 0): (0.1309945201256659-0.5103600712445107j),
 (15, 15, 0): (0.2660530287255519-0.024435535742006794j)}

Having computed this effect at each body in the array we then compute the contribution of all the bodies as isolated ( as in step 1) induced by new excitation

#### Get the all other potential at each location/body with new excitation
If new excitation is the potential and we have this new excitation **at** each location, how do we go from getting the contribution of this $\phi$ to other location?

- Logical thing seem here is to apply the phi_starj function to the other location with xyz and get new excitation until the 2*times the number of bodies as suggested in the paper. ok! let's just do that

- First thing is to somehow calculate the potential to the neighbor as with the airywave potential earlier but this time its new excitation from lsat time.

- Calculate new `body_potential_at_neighbors` from earlier to loop it back in a while loop. The potential at other location this time is not going to be the airywave potential calculation but just the  calculation function `phi_j_star` but keep new potential at each neigbors in the format `body_potential_at_neighbors` 



In [521]:
body_potential_at_neighbors = {body:{nbros : phi_j_star(new_excitation[xyz],thetas[loc_bodies[body]][nbros],nbros,xyz,z,wave_num)
                                              for nbros in neighbors}for xyz,body in loc_to_body.items() 
                                          }

the multiplier for (0, 0, 0)(10, 10, 0) and 0.7853981633974483 is (0.0006380785685019252-0.0005605442829208494j)
the multiplier for (5, 5, 0)(10, 10, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (15, 15, 0)(10, 10, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (10, 10, 0)(0, 0, 0) and 0.7853981633974483 is (0.0006380785685019252-0.0005605442829208494j)
the multiplier for (5, 5, 0)(0, 0, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (15, 15, 0)(0, 0, 0) and 0.7853981633974483 is (1.164007847899644e-05-2.1844294383065847e-05j)
the multiplier for (0, 0, 0)(5, 5, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (10, 10, 0)(5, 5, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (15, 15, 0)(5, 5, 0) and 0.7853981633974483 is (0.0006380785685019252-0.0005605442829208494j)
the 

In [522]:
                           
body_potential_at_neighbors

{FloatingBody(mesh=sphere_1776, dofs={Heave}, name=10_10_0): {(0,
   0,
   0): (6.561970253764026e-05-0.00044267529851284667j),
  (10, 10, 0): 0,
  (5, 5, 0): (0.007463602981185093-0.013419774573960876j),
  (15, 15, 0): (0.007463602981185093-0.013419774573960876j)},
 FloatingBody(mesh=sphere_1782, dofs={Heave}, name=0_0_0): {(0, 0, 0): 0,
  (10, 10, 0): (-0.00020137343839513408-0.00010459417929514332j),
  (5, 5, 0): (-0.005200226776483992-0.005795137830016248j),
  (15, 15, 0): (-6.566584636778234e-06-7.827958613980996e-07j)},
 FloatingBody(mesh=sphere_1788, dofs={Heave}, name=5_5_0): {(0,
   0,
   0): (-0.00167280144723746-0.015264254138175308j),
  (10, 10, 0): (-0.00167280144723746-0.015264254138175308j),
  (5, 5, 0): 0,
  (15, 15, 0): (-0.00020249462428380632-0.0003990780530306401j)},
 FloatingBody(mesh=sphere_1794, dofs={Heave}, name=15_15_0): {(0,
   0,
   0): (2.5631010977837945e-06-6.0961722347005115e-06j),
  (10, 10, 0): (0.007004374140459278-0.0034006946196640277j),
  (5, 5, 0)

### Now we basically while loop the whole thing until the max-iteration

In [523]:
N_bodies = 4
max_iteration = 2*N_bodies #(dead or alive lol)

body_potential_at_neighbors = {body:(dict(zip(body_neighbors_locs[body], 
                                       airy_waves_potential(np.array(body_neighbors_locs[body]),diff_problems[body])))) for body in bodies}


iterate = 0
while iterate<max_iteration:
    # def get_all_other_phi(body_potential_at_neighbors):
    all_other_phi_each_loc = {xyz:{loc_bodies.get(d):k.get(xyz,0) for d,k in body_potential_at_neighbors.items()} for xyz in xyzees}
    print('iteration')
   # print(all_other_phi_each_loc)
    thetas = {k:{s:theta_ij(k,s) for s,m in v.items()} for k,v in all_other_phi_each_loc.items()}
    phi_starj = {xyz:{nbros:phi_j_star(all_other_phi_each_loc[xyz][nbros],thetas[xyz][nbros],nbros,xyz,z,wave_num) for nbros in neighbors} for xyz in xyzees}
    
    new_excitation = get_phistarj_sum(phi_starj,xyzees)
    # look at the new excitation amplitude and reject if the amplitude is bigger than the last two
    
#     print(f"excitation for {iterate}")
    print(new_excitation)
#     print("\n")
    
    
    body_potential_at_neighbors = {body:{nbros : airy_waves_potential(np.array(body_neighbors_locs[body]),diff_problems[body])+ phi_j_star(new_excitation[xyz],thetas[loc_bodies[body]][nbros],nbros,xyz,z,wave_num) 
                                              for nbros in neighbors} for xyz,body in loc_to_body.items()}
    
   # print(new_excitation)
    
    iterate+=1

new_potential = get_phistarj_sum(phi_starj,xyzees)
    

iteration
the multiplier for (0, 0, 0)(10, 10, 0) and 0.7853981633974483 is (0.0006380785685019252-0.0005605442829208494j)
the multiplier for (5, 5, 0)(10, 10, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (15, 15, 0)(10, 10, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (10, 10, 0)(0, 0, 0) and 0.7853981633974483 is (0.0006380785685019252-0.0005605442829208494j)
the multiplier for (5, 5, 0)(0, 0, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (15, 15, 0)(0, 0, 0) and 0.7853981633974483 is (1.164007847899644e-05-2.1844294383065847e-05j)
the multiplier for (0, 0, 0)(5, 5, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (10, 10, 0)(5, 5, 0) and 0.7853981633974483 is (0.027270902746527253-0.0102773327331863j)
the multiplier for (15, 15, 0)(5, 5, 0) and 0.7853981633974483 is (0.0006380785685019252-0.0005605442829208

In [524]:
body_potential_at_neighbors 

{FloatingBody(mesh=sphere_1776, dofs={Heave}, name=10_10_0): {(0,
   0,
   0): array([-3.94896247e-04-8.91839378j,  5.15799005e+00-7.27572839j,
          8.57250832e+00+2.45871227j]),
  (10,
   10,
   0): array([0.        -8.91818182j, 5.15792489-7.27528541j,
         8.57245113+2.45907473j]),
  (5,
   5,
   0): array([-0.01011484-8.92976609j,  5.16537748-7.28872055j,
          8.57867348+2.44812849j]),
  (15,
   15,
   0): array([-0.01011484-8.92976609j,  5.16537748-7.28872055j,
          8.57867348+2.44812849j])},
 FloatingBody(mesh=sphere_1782, dofs={Heave}, name=0_0_0): {(0,
   0,
   0): array([5.15792489-7.27528541j, 8.41547671-2.95190086j,
         8.57245113+2.45907473j]),
  (10,
   10,
   0): array([5.15772316-7.27539003j, 8.41550508-2.95212381j,
         8.57260703+2.45890975j]),
  (5,
   5,
   0): array([5.15271335-7.28108568j, 8.41908538-2.95871614j,
         8.57945315+2.45566368j]),
  (15,
   15,
   0): array([5.15791831-7.27528619j, 8.41547519-2.95190723j,
         8.5724

In [525]:
new_potential[(10, 10, 0)]

array([-0.18459976-0.49435362j,  0.40186734-0.34120671j,
        0.33224829-0.27617749j])

In [526]:
# # function to input new potential from plane-wave approximation
# diff_problems = {body:DiffractionProblemIter(body=body, sea_bottom=-np.infty,
#                                       omega=omega, wave_direction=0.) for body in bodies}

# diff_results = {body:solver.solve(problem) for body,problem in diff_problems.items()}

def solve(diff,diff_res, rad_res,new_potential,keep_details=True):
        """Solve the linear potential flow problem.
        Parameters
        ----------
        problem: LinearPotentialFlowProblem
            the problem to be solved
        keep_details: bool, optional
            if True, store the sources and the potential on the floating body in the output object
            (default: True)
        Returns
        -------
        LinearPotentialFlowResult
            an object storing the problem data and its results
        """
        diff_pot = diff_res.potential
       
        rad_pot = rad_res.potential
        potential = new_potential + diff_pot + rad_pot
        rho = 1000
        new_pressure = rho * potential
        # Actually, for diffraction problems: pressure over jω
        #           for radiation problems:   pressure over -ω²
        # The correction is done in `store_force` in the `result` object.

        new_forces = diff.body.integrate_pressure(new_pressure)

#         if not keep_details:
#             result = problem.make_results_container(new_forces)
#         else:
#             result = problem.make_results_container(new_forces, sources, new_potential, new_pressure)
        return new_forces
    

    


In [527]:

new_results = {loc_to_body.get(loc):solve(diff_prob,diff_results[loc_to_body.get(loc)],rad_results[loc_to_body.get(loc)], sum(new_potential[loc])) for loc,diff_prob in loc_diff.items()}
new_results
    

{FloatingBody(mesh=sphere_1776, dofs={Heave}, name=10_10_0): {'Heave': (4719.197540047971-9020.769399313442j)},
 FloatingBody(mesh=sphere_1782, dofs={Heave}, name=0_0_0): {'Heave': (11232.340659931395+12411.700548485385j)},
 FloatingBody(mesh=sphere_1788, dofs={Heave}, name=5_5_0): {'Heave': (9289.754496572032+770.5062910268164j)},
 FloatingBody(mesh=sphere_1794, dofs={Heave}, name=15_15_0): {'Heave': (4050.227994599498-10150.885612289898j)}}

Somehow keep track of the delta and use the last one before it got less than 10e-2??

In [529]:

%%writefile pwaFUNC.py
'''Plane wave approximation stuff slay!'''
def plane_wave(inputs):
    def generate_body(xyz):
        mesh1 = cpt.meshes.predefined.mesh_sphere(radius=2,center=(xyz[0],xyz[1],xyz[2]))
        body = cpt.FloatingBody(mesh1)
        body.add_translation_dof(name='Heave')
        body = body.immersed_part()
        body.name = f'{xyz[0]}_{xyz[1]}_{xyz[2]}'
        return body


    def get_results(problems):
        results = [solver.solve(pb, keep_details = True) for pb in sorted(problems)]
        return results


    #calculate angle theta_ij from centre of one body to other
    def theta_ij(X,Y): 
        x1,y1= X[0],X[1]
        x2,y2 = Y[0], Y[1]

        if x1 ==x2 and y1==y2:
            return 0
        if x2==x1:
            theta = np.pi/2
        else:
            theta = np.arctan((y2-y1)/(x2-x1))
        return theta


    #step 2
    def phi_j_star(phi_ij,theta,X,Y,z,k):

        '''phi_ij is the vector of all the effect at that body from all other bodies'''
        x,y = X[0],X[1]
        xj,yj = Y[0],Y[1]
        if x==xj and y==yj:
            return 0
        multiplier = np.exp((1j*k*(-1*np.abs(x-xj))*np.cos(theta)) + ((-1*np.abs(y-yj))*np.sin(theta)))
        print(f'the multiplier for {X}{Y} and {theta} is {multiplier}')
        res = phi_ij * multiplier #kz = 0 #e^kz = 1
        return res

    #{(10, 10, 0): {(10, 10, 0): 0,
      #(0, 0, 0): (8.415476709952118-2.9519008598532284j),
      #(5, 5, 0): (8.415476709952118-2.9519008598532284j),
      #(15, 15, 0): (8.415476709952118-2.9519008598532284j)

    def get_phistarj_sum(phi_starj,xyzees):
        xyz_phi = {xyz :[] for xyz in xyzees}
        for k,v in phi_starj.items():
            for s,m in v.items():
                print(f"thee value of m = {m}")
                xyz_phi[k].append(m)
            print("next xyz")
        print(xyz_phi.items())
        xyz_phi = {k:sum(v) for k,v in xyz_phi.items()}
        print("Afteer summation ")
        print(xyz_phi.items())
        return xyz_phi


    def solve(diff,diff_res, rad_res,new_potential,keep_details=True):
            """Solve the linear potential flow problem.
            Parameters
            ----------
            problem: LinearPotentialFlowProblem
                the problem to be solved
            keep_details: bool, optional
                if True, store the sources and the potential on the floating body in the output object
                (default: True)
            Returns
            -------
            LinearPotentialFlowResult
                an object storing the problem data and its results
            """
            diff_pot = diff_res.potential

            rad_pot = rad_res.potential
            potential = new_potential + diff_pot + rad_pot
            rho = 1000
            new_pressure = rho * potential
            # Actually, for diffraction problems: pressure over jω
            #           for radiation problems:   pressure over -ω²
            # The correction is done in `store_force` in the `result` object.

            new_forces = diff.body.integrate_pressure(new_pressure)

    #         if not keep_details:
    #             result = problem.make_results_container(new_forces)
    #         else:
    #             result = problem.make_results_container(new_forces, sources, new_potential, new_pressure)
            return new_forces

    p = 0
    omega = 1.1
    rho = 850 # density of our special material
    wave_amp = 1
    wave_num =  1.0/9.81


    N_bodies = 4
    max_iteration = 2*N_bodies #(dead or alive lol)

    # body_potential_at_neighbors = {body:(dict(zip(body_neighbors_locs[body], 
    #                                       airy_waves_potential(np.array(body_neighbors_locs[body]),diff_problems[body])))) for body in bodies}

    xyzees = {(0,0,0),(5,5,0),(10,10,0),(15,15,0)}
    
    bodies = [generate_body(xyz) for xyz in xyzees ]

    neighbors = {(0,0,0):[(5,5,0),(10,10,0),(15,15,0)],  #so bad..need to write a funky func for it
                (10,10,0):[(0,0,0),(5,5,0),(15,15,0)],
                 (5,5,0):[(0,0,0),(10,10,0),(15,15,0)],
                 (15,15,0):[(0,0,0),(10,10,0),(5,5,0)]     
                }
    loc_bodies = {body:xyz for xyz,body in zip(xyzees,bodies)}
    loc_to_body = {xyz:body for xyz,body in zip(xyzees,bodies)}
    solver = cpt.BEMSolver()


    diff_problems = {body:cpt.DiffractionProblem(body=body, sea_bottom=-np.infty,
                                          omega=omega, wave_direction=0.) for body in bodies}

    diff_loc= {generate_body(loc):loc for loc in xyzees }

    loc_diff = {loc_bodies.get(body):diff for body,diff in diff_problems.items() }

    rad_problems = {body: cpt.RadiationProblem(body=body, sea_bottom=-np.infty,
                                          omega=omega) for body in bodies}

    diff_results = {body:solver.solve(problem) for body,problem in diff_problems.items()}
    rad_results = {body:solver.solve(problem) for body,problem in rad_problems.items()}

    body_neighbors_locs = {body:neighbors.get(loc_bodies.get(body)) for body in bodies}





    body_potential_at_neighbors = {body:{nbros : airy_waves_potential(np.array(body_neighbors_locs[body]),diff_problems[body])
                                                  for nbros in neighbors} for xyz,body in loc_to_body.items()}

    all_other_phi_each_loc = {xyz:{loc_bodies.get(d):k.get(xyz,0) for d,k in body_potential_at_neighbors.items()} for xyz in xyzees}


    thetas = {k:{s:theta_ij(k,s) for s,m in v.items()} for k,v in all_other_phi_each_loc.items()}

    z = 0
    phi_starj = {xyz:{nbros:phi_j_star(all_other_phi_each_loc[xyz][nbros],thetas[xyz][nbros],nbros,xyz,z,wave_num) for nbros in neighbors} for xyz in xyzees}

    new_excitation = get_phistarj_sum(phi_starj,xyzees)
    body_potential_at_neighbors = {body:{nbros : airy_waves_potential(np.array(body_neighbors_locs[body]),diff_problems[body])
                                                  for nbros in neighbors} for xyz,body in loc_to_body.items()}
    iterate = 0
    while iterate<max_iteration:
        # def get_all_other_phi(body_potential_at_neighbors):
        all_other_phi_each_loc = {xyz:{loc_bodies.get(d):k.get(xyz,0) for d,k in body_potential_at_neighbors.items()} for xyz in xyzees}
        print('iteration')
       # print(all_other_phi_each_loc)
        thetas = {k:{s:theta_ij(k,s) for s,m in v.items()} for k,v in all_other_phi_each_loc.items()}
        phi_starj = {xyz:{nbros:phi_j_star(all_other_phi_each_loc[xyz][nbros],thetas[xyz][nbros],nbros,xyz,z,wave_num) for nbros in neighbors} for xyz in xyzees}

        new_excitation = get_phistarj_sum(phi_starj,xyzees)
        # look at the new excitation amplitude and reject if the amplitude is bigger than the last two

    #     print(f"excitation for {iterate}")
        print(new_excitation)
    #     print("\n")


        body_potential_at_neighbors = {body:{nbros : airy_waves_potential(np.array(body_neighbors_locs[body]),diff_problems[body])+ phi_j_star(new_excitation[xyz],thetas[loc_bodies[body]][nbros],nbros,xyz,z,wave_num) 
                                                  for nbros in neighbors} for xyz,body in loc_to_body.items()}

       # print(new_excitation)

        iterate+=1

    new_potential = get_phistarj_sum(phi_starj,xyzees)

    new_results = {loc_to_body.get(loc):solve(diff_prob,diff_results[loc_to_body.get(loc)],rad_results[loc_to_body.get(loc)], sum(new_potential[loc])) for loc,diff_prob in loc_diff.items()}
    return new_results

Overwriting pwaFUNC.py
